## Read the pre-computed global/local keys file
This file, called `key_analysis.txt`, was computed by calling `justkeydding v0.12.0` directly from the bash terminal

```bash
python -m justkeydding --json NAME_OF_THE_FILE
```

The file is structured in pairs of lines. The first line of each pair contains the name of the file. The second one, the `json` output given by `justkeydding v0.12.0`.

In [23]:
import json
import pandas as pd

In [17]:
with open('key_analysis.txt') as fd:
    lines = fd.readlines()

Turning the lines into a dictionary

In [50]:
analyses = {name.strip(): json.loads(analysis.strip()) for name, analysis in zip(lines[::2], lines[1::2])}

After the json conversion, all the time indexes were encoded as strings. Converting them to float.

In [51]:
for file, analysis in analyses.items():    
    analysis['local_keys'] = {float(k): v for k, v in analysis['local_keys'].items()}    

Now I can create a DataFrame off every dictionary.

In [52]:
df = pd.DataFrame(analyses)

In [53]:
df

,1_op12_1_aligned.musicxml,1_op12_1_corrected.musicxml,1_op12_1_omr.musicxml,1_op12_1_revised.musicxml,1_op12_2_aligned.musicxml,1_op12_2_corrected.musicxml,1_op12_2_omr.musicxml,1_op12_2_revised.musicxml,1_op12_3_aligned.musicxml,1_op12_3_corrected.musicxml,...,FP_op81_2_omr.musicxml,FP_op81_2_revised.musicxml,FP_op81_3_aligned.musicxml,FP_op81_3_corrected.musicxml,FP_op81_3_omr.musicxml,FP_op81_3_revised.musicxml,FP_op81_4_aligned.musicxml,FP_op81_4_corrected.musicxml,FP_op81_4_omr.musicxml,FP_op81_4_revised.musicxml
global_key,Eb,Eb,c,Eb,G,G,G,G,Bb,Bb,...,a,a,e,e,e,e,Eb,Eb,Eb,Eb
local_keys,"{0.0: 'Ab', 2.0: 'Ab', 3.5: 'Ab', 4.0: 'Ab', 8...","{0.0: 'Ab', 2.0: 'Ab', 3.5: 'Ab', 4.0: 'Ab', 8...","{0.0: 'Ab', 2.0: 'Ab', 3.5: 'Ab', 4.0: 'Ab', 8...","{0.0: 'Ab', 2.0: 'Ab', 3.5: 'Ab', 4.0: 'Ab', 8...","{0.0: 'g', 0.5: 'g', 1.0: 'g', 1.5: 'g', 2.0: ...","{0.0: 'g', 0.5: 'g', 1.0: 'g', 1.5: 'g', 2.0: ...","{0.0: 'g', 0.5: 'g', 1.0: 'g', 2.0: 'g', 2.5: ...","{0.0: 'g', 0.5: 'g', 1.0: 'g', 1.5: 'g', 2.0: ...","{0.0: 'Bb', 1.0: 'Bb', 1.5: 'Bb', 2.0: 'Bb', 2...","{0.0: 'Bb', 1.0: 'Bb', 1.5: 'Bb', 2.0: 'Bb', 2...",...,"{0.0: 'A', 0.5: 'A', 0.75: 'A', 1.0: 'A', 1.25...","{0.0: 'A', 0.5: 'A', 0.75: 'A', 1.0: 'A', 1.25...","{0.0: 'e', 0.5: 'e', 1.0: 'e', 1.5: 'e', 2.0: ...","{0.0: 'e', 0.5: 'e', 1.0: 'e', 1.5: 'e', 2.0: ...","{0.0: 'e', 0.5: 'e', 1.0: 'e', 1.5: 'e', 2.0: ...","{0.0: 'e', 0.5: 'e', 1.0: 'e', 1.5: 'e', 2.0: ...","{0.0: 'Ab', 2.0: 'Ab', 4.0: 'Ab', 7.0: 'Ab', 8...","{0.0: 'Ab', 2.0: 'Ab', 4.0: 'Ab', 7.0: 'Ab', 8...","{0.0: 'Eb', 2.0: 'Eb', 4.0: 'Eb', 7.0: 'Eb', 8...","{0.0: 'Ab', 2.0: 'Ab', 4.0: 'Ab', 7.0: 'Ab', 8..."


In [48]:
for file, analysis in analyses.items():
    df = pd.DataFrame(analysis)
    analyses[file] = df

{'1_op12_1_aligned.musicxml':        global_key local_keys
 0.0            Eb         Ab
 2.0            Eb         Ab
 3.5            Eb         Ab
 4.0            Eb         Ab
 8.5            Eb         Eb
 ...           ...        ...
 1155.0         Eb         Eb
 1155.5         Eb         Eb
 1156.0         Eb         Eb
 1160.0         Eb         Eb
 1164.0         Eb         Eb
 
 [1722 rows x 2 columns],
 '1_op12_1_corrected.musicxml':        global_key local_keys
 0.0            Eb         Ab
 2.0            Eb         Ab
 3.5            Eb         Ab
 4.0            Eb         Ab
 8.5            Eb         Eb
 ...           ...        ...
 1155.0         Eb         Eb
 1155.5         Eb         Eb
 1156.0         Eb         Eb
 1160.0         Eb         Eb
 1164.0         Eb         Eb
 
 [1722 rows x 2 columns],
 '1_op12_1_omr.musicxml':        global_key local_keys
 0.0             c         Ab
 2.0             c         Ab
 3.5             c         Ab
 4.0             c 